In [1]:
import pickle
import os
from glob import glob
import random

import shutil
# root = "/home/j-j10b102/166.약품식별_인공지능_개발을_위한_경구약제_이미지_데이터/01.데이터/1.Training/{}/단일경구약제_5000종/"
# /home/j-j10b102/166.약품식별_인공지능_개발을_위한_경구약제_이미지_데이터/01.데이터/1.Training/원천데이터
root = "/home/j-j10b102/166.약품식별_인공지능_개발을_위한_경구약제_이미지_데이터/01.데이터/1.Training/{}/단일경구약제_5000종/"
print(root.format("name"))
with open("./json_dict.pkl", "rb") as f:
    json_dict = pickle.load(f)
json_dict["200209900"]

import pandas as pd
from copy import deepcopy

df = pd.read_csv("./code2idx.csv")



/home/j-j10b102/166.약품식별_인공지능_개발을_위한_경구약제_이미지_데이터/01.데이터/1.Training/name/단일경구약제_5000종/


In [2]:
json_dict["201301591"]

['K-030590', '이지엔6이브연질캡슐 200mg', 'TS_66_단일']

In [3]:
df[df["item_seq"] == 199303108]

,item_seq,idx,drug_n,name,group
2515,199303108,2515,K-004378,타이레놀정500mg,TS_3_단일


In [4]:
def convert_to_yolo(x, y, width, height, image_width, image_height):
    # 객체의 중심 좌표 계산
    center_x = x + width / 2
    center_y = y + height / 2
    
    # 좌표와 너비, 높이를 이미지 크기로 정규화
    normalized_center_x = center_x / image_width
    normalized_center_y = center_y / image_height
    normalized_width = width / image_width
    normalized_height = height / image_height
    
    # Yolo 포맷으로 변환된 값을 반환
    return normalized_center_x, normalized_center_y, normalized_width, normalized_height

In [5]:
def train_validation_split(item_seq):

    
 

    item = df[df["item_seq"] == item_seq]
    print("item: ",item)

    group_folders = []

    image_group_folder = item["group"].iloc[0]
    

    label_group_folder = image_group_folder.split("_")

    label_group_folder[0] = "TL"

    label_group_folder = "_".join(label_group_folder)

    group_folders.append(image_group_folder)

    group_folders.append(label_group_folder)
    
    
    
    print("group_folders: ", group_folders)
    import random

    arr = {}
    
    for idx, group_folder in enumerate(group_folders):

        drug_n_folder = item["drug_n"].iloc[-1]
        data = {}
        if idx == 0:
            data["drug_n_folder"] = drug_n_folder
            data["path"] = "원천데이터"
            data["extension"] = "png"
            data["dtype"] = "images"
            
        else:
            data["drug_n_folder"] = drug_n_folder+"_json"
            data["path"] = "라벨링데이터"
            data["extension"] = "json"
            data["dtype"] = "labels"
        data["group_folder"] = group_folder
        path = root.format(data["path"])
        path = path+data["group_folder"]+"/"+data["drug_n_folder"]
        files = [os.path.splitext(file)[0].split("/")[-1] for file in glob(path+"/*") if os.path.isfile(file)]
        
        files.sort()
        
        data["files"] = files
        arr[idx] = data

    random.shuffle(arr[0]["files"])
    
#     print(arr[0]["files"])
    
    arr[1]["files"] = deepcopy(arr[0]["files"])
#     print("-----")
#     print(arr[1]["files"])
    
    for key, data in arr.items():
        path = root.format(data["path"])
        path = path+data["group_folder"]+"/"+data["drug_n_folder"]
        
        train_folder = path+"/"+data["dtype"]+"/train"
        val_folder = path+"/"+data["dtype"]+"/val"
        
        if not os.path.exists(train_folder):
            os.makedirs(train_folder)
            
        if not os.path.exists(val_folder):
            os.makedirs(val_folder)
            
        files = data["files"]
        
        total_size = len(files)
        train_size = int(0.7 * total_size)
        
        trains = files[:]
        vals = files[train_size:]
        print("len(trains): ", len(trains))
        print("len(vals): ", len(vals))
        
        if key == 0:

            for filename in trains:
                src_file = path + "/"+filename+"."+data["extension"]
                
                dst_file = train_folder+"/"+filename+"."+data["extension"]
                
                shutil.copy(src_file, dst_file)

            for filename in vals:
                src_file = path + "/"+filename+"."+data["extension"]
                

                dst_file = val_folder+"/"+filename+"."+data["extension"]

                shutil.copy(src_file,dst_file)
            
        
        else:
            
            import json
            
            
            for filename in trains:
                src_file = path + "/"+filename+"."+data["extension"]
                with open(src_file, "rb") as f:
                    json_data = json.load(f)
                idx = item["idx"].iloc[0]
                
                height = json_data["images"][0]["height"]
                width = json_data["images"][0]["width"]
                
                x,y,w,h = json_data["annotations"][0]["bbox"]
            
                center_x, center_y, normalized_w, normalized_h = convert_to_yolo(x,y,w,h,width, height)
            
#                 filename, _ = os.path.splitext(src_file)
#                 filename = filename.split("/")[-1]
                
                dst_file = train_folder+"/"+filename+".txt"
                
                
                
                with open(dst_file, "w") as f:
                    f.write(str(idx)+" ")
                    f.write(str(center_x)+" ")
                    f.write(str(center_y)+" ")
                    f.write(str(normalized_w)+" ")
                    f.write(str(normalized_h)+" ")
                    
            for filename in vals:
                src_file = path + "/"+filename+"."+data["extension"]
                with open(src_file, "rb") as f:
                    json_data = json.load(f)
                
                idx = item["idx"].iloc[0]
                
                height = json_data["images"][0]["height"]
                width = json_data["images"][0]["width"]
                
                x,y,w,h = json_data["annotations"][0]["bbox"]
            
                center_x, center_y, normalized_w, normalized_h = convert_to_yolo(x,y,w,h,width, height)
            
#                 filename, _ = os.path.splitext(src_file)
#                 filename = filename.split("/")[-1]
                
                dst_file = val_folder+"/"+filename+".txt"
                
                with open(dst_file, "w") as f:
                    f.write(str(idx)+" ")
                    f.write(str(center_x)+" ")
                    f.write(str(center_y)+" ")
                    f.write(str(normalized_w)+" ")
                    f.write(str(normalized_h)+" ")
                

In [6]:
# train_validation_split(item_seqs[0])

# item_seqs = [199303108,200400485 ,201500041,200300406, 199900926 ,199601520 , 198800190 ,200901181 , 200108429 ,198301005,200301805,201403321 ]
item_seqs = [199303108,200400485,201500041,200300406,200301805,201403321   ]
# train_validation_split(item_seqs[0])
for item_seq in item_seqs:

    train_validation_split(item_seq)
    print("----")

item:         item_seq   idx    drug_n        name    group
2515  199303108  2515  K-004378  타이레놀정500mg  TS_3_단일
group_folders:  ['TS_3_단일', 'TL_3_단일']
len(trains):  1296
len(vals):  389
len(trains):  1296
len(vals):  389
----
item:         item_seq   idx    drug_n  \
2659  200400485  2659  K-014267   

                                                   name     group  
2659  이지엔6프로연질캡슐(덱시부프로펜)(수출명:DAUFENSoftCapsuleSPEEDFEN)  TS_40_단일  
group_folders:  ['TS_40_단일', 'TL_40_단일']
len(trains):  324
len(vals):  98
len(trains):  324
len(vals):  98
----
item:         item_seq   idx    drug_n          name     group
1224  201500041  1224  K-035073  티지피파모티딘정10mg  TS_74_단일
group_folders:  ['TS_74_단일', 'TL_74_단일']
len(trains):  216
len(vals):  65
len(trains):  216
len(vals):  65
----
item:        item_seq  idx    drug_n           name    group
109  200300406  109  K-012769  닥터베아제정 80mg/병  TS_7_단일
group_folders:  ['TS_7_단일', 'TL_7_단일']
len(trains):  1296
len(vals):  389
len(trains):  1296
len(vals